In [1]:
import pandas as pd
import os
import numpy
import pyLDAvis
import pyLDAvis.gensim
import numpy as np
import pandas as pd
import re

C:\Users\ragha\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\ragha\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv(r"C:\Users\ragha\Downloads\lda_preds.csv")
df

,Video path,Caption,Prediction_from_lda
0,['/home/shaunaks/cls_data_1_min/encoded_videos...,a video of a bridge that is over a highway. a...,NaN
1,['/home/shaunaks/cls_data_1_min/encoded_videos...,a car is driving down a road and a man is tal...,NaN
2,['/home/shaunaks/cls_data_1_min/encoded_videos...,a person is walking up a staircase and then a...,NaN
3,['/home/shaunaks/cls_data_1_min/encoded_videos...,a man is squatting down and then he picks up ...,NaN
4,['/home/shaunaks/cls_data_1_min/encoded_videos...,a man in a knight's armor is talking to anoth...,NaN
...,...,...,...
326,['/home/shaunaks/cls_data_1_min/encoded_videos...,a man is sitting at a table and talking to th...,NaN
327,['/home/shaunaks/cls_data_1_min/encoded_videos...,a man is holding a box that is being held by ...,NaN
328,['/home/shaunaks/cls_data_1_min/encoded_videos...,a man in a wheelchair is sitting in a room an...,NaN
329,['/home/shaunaks/cls_data_1_min/encoded_videos...,a little girl is laying down and a cat is lay...,NaN


In [3]:
df['Caption'] = \
df['Caption'].map(lambda x: re.sub('[,\.!?]', '', x))
df['Caption'] = \
df['Caption'].map(lambda x: x.lower())
df['Caption'].head()

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
C:\Users\ragha\AppData\Local\Temp\ipykernel_3640\1509625820.py:2: DeprecationWarning: invalid escape sequence \.
  df['Caption'].map(lambda x: re.sub('[,\.!?]', '', x))


0     a video of a bridge that is over a highway a ...
1     a car is driving down a road and a man is tal...
2     a person is walking up a staircase and then a...
3     a man is squatting down and then he picks up ...
4     a man in a knight's armor is talking to anoth...
Name: Caption, dtype: object

In [4]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = df.Caption.values.tolist()
data_words = list(sent_to_words(data))
data_words = remove_stopwords(data_words)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please import `triu` from the `scipy.linalg` namespace; the `scipy.linalg.special_matrices` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.linalg.special_matrices import triu
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ragha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import gensim.corpora as corpora
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
corpus[0]

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 1),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1)]

In [9]:
from pprint import pprint
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.153*"man" + 0.026*"talking" + 0.024*"another" + 0.023*"woman" + '
  '0.019*"standing" + 0.017*"shown" + 0.017*"camera" + 0.016*"gun" + '
  '0.016*"holding" + 0.015*"front"'),
 (1,
  '0.100*"man" + 0.036*"fast" + 0.034*"furious" + 0.023*"woman" + '
  '0.023*"standing" + 0.021*"talking" + 0.018*"walks" + 0.017*"car" + '
  '0.015*"shown" + 0.015*"person"'),
 (2,
  '0.168*"man" + 0.077*"talking" + 0.057*"woman" + 0.039*"sitting" + '
  '0.033*"another" + 0.022*"group" + 0.020*"men" + 0.020*"car" + '
  '0.018*"people" + 0.016*"shown"'),
 (3,
  '0.060*"man" + 0.036*"camera" + 0.027*"looking" + 0.022*"shown" + '
  '0.018*"woman" + 0.018*"boy" + 0.015*"scene" + 0.014*"person" + '
  '0.013*"movie" + 0.013*"room"'),
 (4,
  '0.055*"man" + 0.047*"talking" + 0.037*"car" + 0.032*"sitting" + '
  '0.026*"person" + 0.025*"woman" + 0.020*"walks" + 0.019*"driving" + '
  '0.016*"standing" + 0.015*"away"'),
 (5,
  '0.066*"man" + 0.051*"woman" + 0.028*"standing" + 0.022*"looking" + '
  '0.022*"talk

In [7]:
# lda_model.get_document_topics(id2word.doc2bow(data_words[0]), minimum_probability=0.0)

In [10]:
new_df = pd.read_csv(r"C:\Users\ragha\Downloads\predictions_test.csv")
new_df

,Video path,Prediction,Target
0,['/home/shaunaks/cls_data_1_min/encoded_videos...,explicit,explicit
1,['/home/shaunaks/cls_data_1_min/encoded_videos...,explicit,explicit
2,['/home/shaunaks/cls_data_1_min/encoded_videos...,non_explicit,explicit
3,['/home/shaunaks/cls_data_1_min/encoded_videos...,explicit,explicit
4,['/home/shaunaks/cls_data_1_min/encoded_videos...,non_explicit,explicit
...,...,...,...
326,['/home/shaunaks/cls_data_1_min/encoded_videos...,non_explicit,non_explicit
327,['/home/shaunaks/cls_data_1_min/encoded_videos...,non_explicit,non_explicit
328,['/home/shaunaks/cls_data_1_min/encoded_videos...,non_explicit,non_explicit
329,['/home/shaunaks/cls_data_1_min/encoded_videos...,non_explicit,non_explicit


In [11]:
dataset = pd.DataFrame(columns=['topics', 'ans'])
for i in range(331):
    l1 = []
    temp = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
    for j in range(len(temp)):
        l1.append(temp[j][1])
    if new_df.iloc[i, 2] == 'explicit': 
        a = 1
    else:
        a = 0
    dataset.loc[i] = [l1, a]
dataset

,topics,ans
0,"[0.0066687292, 0.0066682054, 0.34373125, 0.006...",1
1,"[0.983923, 0.001786252, 0.001786524, 0.0017860...",1
2,"[0.002223, 0.002222769, 0.4576297, 0.002222651...",1
3,"[0.0023265914, 0.27683693, 0.70455164, 0.00232...",1
4,"[0.98162836, 0.0020412568, 0.002041555, 0.0020...",1
...,...,...
326,"[0.0037043625, 0.0037041565, 0.96666026, 0.003...",0
327,"[0.0021746336, 0.0021745563, 0.37072462, 0.002...",0
328,"[0.003572385, 0.0035720528, 0.9678505, 0.00357...",0
329,"[0.0023261434, 0.0023261276, 0.97906417, 0.002...",0


In [12]:
X = dataset.topics.to_list()
y = dataset.ans.to_list()

In [13]:
# from gensim.models.coherencemodel import CoherenceModel

# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('nCoherence Score: ', coherence_lda)

In [14]:
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
p

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.013465  0.041765       1        1  54.444206
0      0.055157  0.004088       2        1  10.881506
7      0.044143  0.018540       3        1   7.830796
8      0.031091 -0.030259       4        1   6.582737
1      0.051581 -0.010348       5        1   4.630851
9      0.016306  0.033315       6        1   4.576215
6     -0.122916  0.052495       7        1   3.514337
4     -0.009991  0.049328       8        1   3.063723
5     -0.038180 -0.077179       9        1   2.294693
3     -0.040658 -0.081746      10        1   2.180936, topic_info=        Term         Freq        Total Category  logprob  loglift
5        man  1436.000000  1436.000000  Default  30.0000  30.0000
27     woman   494.000000   494.000000  Default  29.0000  29.0000
9    sitting   325.000000   325.000000  Default  28.0000  28.0000
10   talking   574.000000   574.000000  Default  27.0000  27.0000
2        car   203.000000   203.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
82    people     1.920053   169.920183  Topic10  -4.7993  -0.6576
75     group     1.898733   205.686013  Topic10  -4.8105  -0.8597
72    around     1.515443    78.244227  Topic10  -5.0359  -0.1187
87  standing     1.607803   180.632924  Topic10  -4.9768  -0.8962
94    street     1.370296    65.447248  Topic10  -5.1366  -0.0408

[663 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
633       1  0.384815    actor
633       5  0.192407    actor
633       8  0.192407    actor
767       4  0.518561    angel
0         1  0.748161  another
...     ...       ...      ...
410       5  0.044809    young
410       6  0.044809    young
410       7  0.089617    young
410       9  0.044809    young
410      10  0.089617    young

[1274 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 8, 9, 2, 10, 7, 5, 6, 4])

In [65]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
clf = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42).fit(X_train, y_train)

In [66]:
y_pred = clf.predict(X_train)
print("Train accuracy: ", accuracy_score(y_train, y_pred))
print("Train F1: ", f1_score(y_train, y_pred))

Train accuracy:  0.7338709677419355
Train F1:  0.6373626373626373


In [67]:
y_pred = clf.predict(X_test)
print("Test accuracy: ", accuracy_score(y_test, y_pred))
print("Test F1: ", f1_score(y_test, y_pred))

Test accuracy:  0.7108433734939759
Test F1:  0.6363636363636364
